In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pickle
import numpy as np

In [2]:
df = pd.read_csv('/home/nikolay/EMAI/Ljublajna/Explainable-AI-Project/user_study_dataset/models/data.csv')
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,1,55.0,0,0,1,45.20,9.0,280,1
1,0,14.0,0,0,0,20.13,6.1,126,0
2,1,64.0,1,0,0,33.87,9.0,140,1
3,0,72.0,0,0,0,22.79,7.0,140,1
4,0,36.0,0,0,0,40.00,5.7,260,1


In [3]:
rf = RandomForestClassifier(n_estimators=30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(df.drop('diabetes', axis=1), df['diabetes'], test_size=0.2, random_state=42)

rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=30, random_state=42)

In [6]:
preds = rf.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, preds))

Accuracy:  0.8850488354620586


In [19]:
cf_sample = X_test.iloc[0]

In [33]:
cf_sample_df = pd.DataFrame(cf_sample).transpose()

In [40]:
cf_sample_df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level
6113,1.0,54.0,0.0,0.0,1.0,35.02,7.0,160.0


In [34]:
cf_sample_df.shape

(1, 8)

In [19]:
from carla import MLModelCatalog
from carla.data.catalog import OnlineCatalog
from carla.recourse_methods import GrowingSpheres

# load a catalog dataset
data_name = "adult"
dataset = OnlineCatalog(data_name)

# load artificial neural network from catalog
model = MLModelCatalog(dataset, "ann")

In [20]:
dataset.df

,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.301370,0.044131,0.800000,0.021740,0.0,...,0.0,1.0,1.0,1.0,0.0
1,0.452055,0.048052,0.800000,0.000000,0.0,...,0.0,1.0,0.0,1.0,0.0
2,0.287671,0.137581,0.533333,0.000000,0.0,...,1.0,1.0,1.0,1.0,1.0
3,0.493151,0.150486,0.400000,0.000000,0.0,...,1.0,0.0,0.0,1.0,1.0
4,0.150685,0.220635,0.800000,0.000000,0.0,...,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
48827,0.301370,0.137428,0.800000,0.000000,0.0,...,0.0,1.0,1.0,0.0,1.0
48828,0.643836,0.209130,0.533333,0.000000,0.0,...,0.0,0.0,1.0,1.0,1.0
48829,0.287671,0.245379,0.800000,0.000000,0.0,...,0.0,1.0,0.0,1.0,1.0
48830,0.369863,0.048444,0.800000,0.054551,0.0,...,0.0,0.0,1.0,1.0,1.0


In [23]:
# get factuals from the data to generate counterfactual examples
factuals = dataset.df[:1]

# load a recourse model and pass black box model
gs = GrowingSpheres(model)

# generate counterfactual examples
counterfactuals = gs.get_counterfactuals(factuals)

/home/nikolay/EMAI/Ljublajna/Explainable-AI-Project/.venv/lib/python3.7/site-packages/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  f"cannot re-order features for non dataframe input: {type(x)}"


In [17]:
model.predict_proba(counterfactuals)

array([[0.18951373, 0.81048626]], dtype=float32)

In [18]:
model.predict_proba(factuals)

array([[0.69982654, 0.3001735 ]], dtype=float32)